In [1]:
import pandas as pd
from bson.objectid import ObjectId
from pymongo import MongoClient
import json

In [2]:
items = pd.read_excel("Sit_in_Menu.xlsx")
client = MongoClient('mongodb://127.0.0.1:27017/tasteofindia')

In [3]:
items['SpiceLevel'] = ""
items['IsFeatured'] = False
items['IsAvailable'] = True
items['Description'] = items.Name + " Description"
items.drop('Touchscreen', axis=1, inplace=True)
items.drop('Tax', axis=1, inplace=True)
items.drop('Printer', axis=1, inplace=True)
items.drop('Unnamed: 6', axis=1, inplace=True)

In [4]:
items.dropna(inplace=True)

In [5]:
# Build categories
client.tasteofindia.categories.drop()
for departement in items.Department.unique():
    d = {'isAvailable': True, 'name': departement, 'description': departement, '_v': 0}
    client.tasteofindia.categories.insert_one(d)

In [6]:
def getCategoryId(name):
    category = client.tasteofindia.categories.find_one({'name': name})
    return category['_id']

In [7]:
def getComplimetaryItemId():
    citems = client.tasteofindia.complimentaryitems.find()
    return [citem['_id'] for citem in citems]

In [8]:
client.tasteofindia.items.drop()
for index, item in items.iterrows():
    d = {
        'defaultSpiceLevel': "", 
        'isFeatured': item['IsFeatured'],
        'isAvailable': item['IsAvailable'],
        'name': item['Name'],
        'description': item['Description'],
        'price': item['Selling'],
        'category': getCategoryId(item['Department'])
    }
    if item['Department'] in ['Chicken Mains', 'Lamb Mains', 'Seafood Mains', 'Vegetable Mains', 'Indo Chinese Main', 'Mutton (Goat)']:
        d['spiceLevels'] = ['Mild', 'Medium', 'Hot']
        d['ComplimentaryItems'] = getComplimetaryItemId()
    client.tasteofindia.items.insert_one(d)

In [9]:
items.Department.unique()

array(['Veg Starters', 'Non-Veg Starters', 'Soups', 'Salad',
       'Chicken Mains', 'Lamb Mains', 'Seafood Mains', 'Rice',
       'Vegetable Mains', 'Indo Chinese Main', 'Mutton (Goat)',
       'Indian Breads', 'Deserts', 'Hot Beverages'], dtype=object)

In [10]:
# rice = {'name': 'Rice', 'price': 0}
# butter_naan = {'name': 'Butter Naan', 'price': 0}
# garlic_naan = {'name': 'Garlic Naan', 'price': 0}

# client.tasteofindia.complimentaryitems.insert_many([rice, butter_naan, garlic_naan])